In [ ]:
def power_config(data, threshold):
    #data = data.dropna()
    avg = data.mean()
    if avg > threshold:
        return 1
    else:
        return 0

In [15]:
import pandas as pd
import math

def main():
    new = True

    main_meter = 'mains1'
    labels = ['mains1','kitchen_outlets_21','kitchen_outlets_22','furnace','washer_dryer_13',
              'washer_dryer_14', 'microwave', 'bathroom_gfi']
    if new is True:
        main1 = pd.read_csv("low_freq/house_3/channel_1.dat", sep=" ", header=None, index_col=0, squeeze=True)
        df = pd.DataFrame(columns=labels, index=main1.index)
        df['mains1'] = main1
       
        ko_21 = pd.read_csv("low_freq/house_3/channel_21.dat", sep=" ", header=None, index_col=0, squeeze=True)
        ko_22 = pd.read_csv("low_freq/house_3/channel_22.dat", sep=" ", header=None, index_col=0, squeeze=True)
      
        df['kitchen_outlets_21'] = ko_21
        df['kitchen_outlets_22'] = ko_22
       
        
        furnace = pd.read_csv("low_freq/house_3/channel_10.dat", sep=" ", header=None, index_col=0, squeeze=True)
        df['furnace'] = furnace
        
        microwave = pd.read_csv("low_freq/house_3/channel_16.dat", sep=" ", header=None, index_col=0, squeeze=True)
        df['microwave'] = microwave
        
        washer_dryer_13 = pd.read_csv("low_freq/house_3/channel_13.dat", sep=" ", header=None, index_col=0, squeeze=True)
        washer_dryer_14 = pd.read_csv("low_freq/house_3/channel_14.dat", sep=" ", header=None, index_col=0, squeeze=True)
        df['washer_dryer_13'] = washer_dryer_13
        df['washer_dryer_14'] = washer_dryer_14
       
        bathroom_gfi = pd.read_csv("low_freq/house_3/channel_20.dat", sep=" ", header=None, index_col=0, squeeze=True)
        df['bathroom_gfi'] = bathroom_gfi
        
        start_row, end_row = ko_21.index[0], ko_21.index[-1]
       
    
        df = df[start_row <= df.index]
        df = df[df.index <= end_row + 3]
        df = df.reset_index(drop=True)
        df.to_csv("house3-nosampling/combine_h3.csv")
    else:
        df = pd.read_csv("house1-nosampling/combine_h1.csv", index_col=0)

In [16]:
main()

In [17]:
combine_h3 = pd.read_csv("house3-nosampling/combine_h3.csv", index_col=0)

In [18]:
combine_h3 = combine_h3.dropna()
combine_h3 = combine_h3.reset_index(drop=True)

In [20]:
house3_threshold = pd.read_csv("house3-nosampling/threshold.csv")

In [21]:
labeled_combine_h3 = pd.DataFrame()

In [22]:
labeled_ko21 = [0 if combine_h3['kitchen_outlets_21'][i] < house3_threshold['kitchen_outlets_21'][0] else 1 for i in range(len(combine_h3))]
labeled_ko22 = [0 if combine_h3['kitchen_outlets_22'][i] < house3_threshold['kitchen_outlets_22'][0] else 1 for i in range(len(combine_h3))]


In [23]:
labeled_washerdryer13 = [0 if combine_h3['washer_dryer_13'][i] < house3_threshold['washer_dryer_13'][0] else 1 for i in range(len(combine_h3))]
labeled_washerdryer24 = [0 if combine_h3['washer_dryer_14'][i] < house3_threshold['washer_dryer_14'][0] else 1 for i in range(len(combine_h3))]

In [24]:
labeled_furnace = [0 if combine_h3['furnace'][i] < house3_threshold['furnace'][0] else 1 for i in range(len(combine_h3))]

In [25]:
labeled_microwave = [0 if combine_h3['microwave'][i] < house3_threshold['microwave'][0] else 1 for i in range(len(combine_h3))]
labeled_bathroomgfi = [0 if combine_h3['bathroom_gfi'][i] < house3_threshold['bathroom_gfi'][0] else 1 for i in range(len(combine_h3))]

In [26]:
labeled_combine_h3['main'] = combine_h3['mains1']

labeled_combine_h3['kitchen_outlets_21'] = labeled_ko21
labeled_combine_h3['kitchen_outlets_22'] = labeled_ko22

labeled_combine_h3['washer_dryer_13'] = labeled_washerdryer13
labeled_combine_h3['washer_dryer_14'] = labeled_washerdryer24

labeled_combine_h3['furnace'] = labeled_furnace
labeled_combine_h3['microwave'] = labeled_microwave
labeled_combine_h3['bathroom_gfi'] = labeled_bathroomgfi

In [29]:
window = 100

sampl = labeled_combine_h3.copy()
rem = len(sampl) % window
if rem > 0:
    sampl = sampl[:-rem]
#type(sampl.values)

In [31]:
import numpy as np
array_values = sampl.values
n = len(array_values)
k = 10  #steps
window = 100 #window
r = math.floor(n/k)
new_array = []
for i in range(r - (window // k) + 1):
    #print(i)
    next_window = array_values[(i*k):(i*k)+window]
    #print(next_window)
    new_array.append(next_window)
new_array = np.asarray(new_array)
tobe_df_arr = new_array.reshape(new_array.shape[0]*new_array.shape[1],new_array.shape[2]) # reshape

temp_df = pd.DataFrame(tobe_df_arr, columns=labeled_combine_h3.columns)

In [32]:
temp_df.to_csv('house3-nosampling/h3_5min.csv')

In [33]:
len(temp_df)

3760100

In [ ]:
temp_df